In [17]:
# Import libraries
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from PIL import Image

# LOAD THE DATASET

In [18]:
import os
import tensorflow as tf

# Load dataset directly from preprocessed 'cleandata' folder
def load_dataset(folder):
    """
    Load a dataset of preprocessed images from the 'cleandata' folder.
    Args:
        folder (str): Path to the folder containing preprocessed images.
    Returns:
        dataset (tf.data.Dataset): Dataset of preprocessed images.
    """
    image_paths = [os.path.join(folder, fname) for fname in os.listdir(folder) if fname.endswith('.jpg') or fname.endswith('.png')]
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)

    # Decode and convert to float32 in [-1, 1] range
    def process_image(image_path):
        img = tf.image.decode_image(tf.io.read_file(image_path), channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)  # Convert to float32 [0, 1]
        img = (img * 2) - 1  # Normalize to [-1, 1] to match model output
        return img

    dataset = dataset.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset



In [19]:


content_dataset = load_dataset("cleandata/augmented_content")
monet_dataset = load_dataset("cleandata/augmented_monet")
vangogh_dataset = load_dataset("cleandata/augmented_vangogh")

In [20]:
content_dataset

<_ParallelMapDataset element_spec=TensorSpec(shape=<unknown>, dtype=tf.float32, name=None)>

# Batch and shuffle the dataset

###  batch(batch_size)

the batch(batch_size) operation groups the datset into batches of the specified size, it takes ur datset and combines indiviual sample into branches of 16 images each 



### prefetch(tf.data.AUTOTUNE)

This operation is the performance optimization technique
while modle is training on one batch , tensorflow simunltaneously prepares the next batch in the background

without this the model would sit idle waiting for the next batch to load prefetching ensures traning runs more smoothly by reducing data loading bottlenecks

In [21]:
batch_size = 16
content_dataset = content_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
vangogh_dataset = vangogh_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
monet_dataset = monet_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Generator Model

### Overview

the build generator function defines a generator netowrk for your neural style transfer project the goal of this generator is to take a content image as input and transform it intoa stylized image that looks like it was painted in the style of vangogh or monet artist


the generator is based on the cycelgan architecture and uses residual block to capture the complex style patterns

In [ ]:
def build_generator():
    """this defines the input shape of the content image which has to be
      converted to some particular style"""
    inputs = layers.Input(shape=(256, 256, 3))

    # Downsample
    """7*7 kernel  with 64 layers
      normilization make the value with mean center as 0 
      so it will be use full when we  are
      using the relu function  for each convolution layer"""
    x = layers.Conv2D(64, (7, 7), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(128, (3, 3), strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(256, (3, 3), strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Residual blocks
    for _ in range(6):
        residual = x
        x = layers.Conv2D(256, (3, 3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.Conv2D(256, (3, 3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Add()([x, residual])

    # Upsample
    x = layers.Conv2DTranspose(128, (3, 3), strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(3, (7, 7), padding='same')(x)
    outputs = layers.Activation('tanh')(x)  # Output in [-1, 1] range

    return models.Model(inputs, outputs)


# Discriminator 

In [23]:
def build_discriminator():
    inputs = layers.Input(shape=(256, 256, 3))

    x = layers.Conv2D(64, (4, 4), strides=2, padding='same')(inputs)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(128, (4, 4), strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(256, (4, 4), strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(512, (4, 4), strides=2, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(1, (4, 4), padding='same')(x)
    outputs = layers.Activation('sigmoid')(x)

    return models.Model(inputs, outputs)

# build generator and discriminator

In [24]:
generator_content_to_style = build_generator()
generator_style_to_content = build_generator()
discriminator_content = build_discriminator()
discriminator_style = build_discriminator()

In [25]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()

In [26]:
def discriminator_loss(real, generated):
    real_loss = cross_entropy(tf.ones_like(real), real)
    generated_loss = cross_entropy(tf.zeros_like(generated), generated)
    return real_loss + generated_loss

def generator_loss(generated):
    return cross_entropy(tf.ones_like(generated), generated)

def cycle_loss(real, cycled):
    return tf.reduce_mean(tf.abs(real - cycled))

# Optimizers

In [27]:
generator_optimizer = optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = optimizers.Adam(2e-4, beta_1=0.5)

# Training Function

In [28]:
# Training loop
def train_step(content_images, style_images):
    with tf.GradientTape(persistent=True) as tape:
        # Generate images
        fake_style = generator_content_to_style(content_images)
        fake_content = generator_style_to_content(style_images)

        # Cycle consistency
        cycled_content = generator_style_to_content(fake_style)
        cycled_style = generator_content_to_style(fake_content)

        # Discriminator outputs
        real_content_output = discriminator_content(content_images)
        fake_content_output = discriminator_content(fake_content)
        real_style_output = discriminator_style(style_images)
        fake_style_output = discriminator_style(fake_style)

        # Compute losses
        gen_content_to_style_loss = generator_loss(fake_style_output)
        gen_style_to_content_loss = generator_loss(fake_content_output)
        total_gen_loss = gen_content_to_style_loss + gen_style_to_content_loss

        disc_content_loss = discriminator_loss(real_content_output, fake_content_output)
        disc_style_loss = discriminator_loss(real_style_output, fake_style_output)
        total_disc_loss = disc_content_loss + disc_style_loss

        # Cycle consistency loss
        total_cycle_loss = cycle_loss(content_images, cycled_content) + cycle_loss(style_images, cycled_style)
        total_gen_loss += 10 * total_cycle_loss  # Weight for cycle loss

    # Apply gradients
    generator_gradients = tape.gradient(total_gen_loss, generator_content_to_style.trainable_variables + generator_style_to_content.trainable_variables)
    discriminator_gradients = tape.gradient(total_disc_loss, discriminator_content.trainable_variables + discriminator_style.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_gradients, generator_content_to_style.trainable_variables + generator_style_to_content.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator_content.trainable_variables + discriminator_style.trainable_variables))


# Training

In [29]:
from tqdm import tqdm

# Training with a progress bar
epochs = 50
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    progress_bar = tqdm(zip(content_dataset, vangogh_dataset), total=len(content_dataset), desc=f"Training Epoch {epoch+1}", unit="batch")

    for content_batch, style_batch in progress_bar:
        train_step(content_batch, style_batch)

    print(f"✅ Epoch {epoch + 1} completed\n")


Epoch 1/50


Training Epoch 1:  63%|██████▎   | 65/103 [1:34:28<55:13, 87.21s/batch]  


✅ Epoch 1 completed

Epoch 2/50


Training Epoch 2:  63%|██████▎   | 65/103 [1:12:00<42:05, 66.47s/batch]


✅ Epoch 2 completed

Epoch 3/50


...

✅ Epoch 4 completed

Epoch 5/50


Training Epoch 5:  63%|██████▎   | 65/103 [1:07:33<39:29, 62.36s/batch]


✅ Epoch 5 completed

Epoch 6/50


Training Epoch 6:   7%|▋         | 7/103 [08:08<1:51:42, 69.82s/batch]


KeyboardInterrupt: 

# Generate Styled Image

In [30]:
from preprocess import load_add_preprocess_image

c:\Users\PUGAZH\Desktop\Projects\DeepLearning\NeuroBrush\brushvenv\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [33]:
# Generate a stylized image
test_content_image = load_add_preprocess_image(r"data\ContentImage\2014-08-02 11_46_18.jpg")
stylized_image = generator_content_to_style(test_content_image[tf.newaxis, ...])
stylized_image = tf.squeeze(stylized_image, axis=0)

In [34]:
from PIL import Image
import numpy as np

# Convert Tensor to NumPy and scale values
image_array = (stylized_image.numpy() * 255).astype(np.uint8)

# Create and show the image
img = Image.fromarray(image_array)
img.show()
